In [877]:
import re
import ast
import numpy as np
import math
from functools import cache, cmp_to_key
from itertools import product
import copy

def read_file_to_string(file_path):
    with open(file_path,'r') as f:
        return f.read()
    
day_22_string = read_file_to_string('day_22_input.txt')
day_22_test = read_file_to_string('day_22_test.txt')
#day_20_test_2 = read_file_to_string('day_20_test_2.txt')
# day_20_test_3 = read_file_to_string('day_20_test_3.txt')
# day_20_test_4 = read_file_to_string('day_20_test_4.txt')
# day_20_test_5 = read_file_to_string('day_20_test_5.txt')

In [878]:
def parse_string(string):
    day_22=string.split('\n')
    day_22=[tuple([tuple([int(coord) for coord in coords.split(',')]) for coords in row.split('~')]) for row in day_22]

    return day_22

In [879]:
input_bricks = parse_string(day_22_string)
test_bricks = parse_string(day_22_test)

In [12]:
def third_brick_compare(brick1,brick2):
    if brick1[0][2] < brick2[0][2]:
        return -1
    elif brick1[0][2] > brick2[0][2]:
        return 1
    else:
        return 0

In [269]:
def elongate_brick(brick):

    for i in range(3):
        if brick[1][i] - brick[0][i] > 0:

            if i == 0:
                return [(j,brick[0][1],brick[0][2]) for j in range(brick[0][i], brick[1][i]+1)]

            elif i == 1:
                return [(brick[0][0],j,brick[0][2]) for j in range(brick[0][i], brick[1][i]+1)]

            else:
                return [(brick[0][0],brick[0][1],j) for j in range(brick[0][i], brick[1][i]+1)]
            
    return [brick[0]]

In [917]:
def is_brick_under(brick_layer1,brick2):

    #check if brick1 is UNDER brick2

    #first - make each brick a list of all the coords it covers.
    elon_brick1_xy = []
    for brick in brick_layer1:
        elon_brick = elongate_brick(brick)
        elon_brick1_xy += [(point[0],point[1]) for point in elon_brick]
    
    elon_brick2 = elongate_brick(brick2)
    elon_brick2_xy = [(point[0],point[1]) for point in elon_brick2]
   
    
    for point in elon_brick2_xy:
        #print(point)
        if point in elon_brick1_xy:
            return True
    
    return False

In [918]:
is_brick_under([((1, 0, 1), (1, 2, 1))],((0, 0, 2), (2, 0, 2)))

True

In [919]:
def remove_brick_gaps(brick_list):
    difference = 0
    for i, brick in enumerate(brick_list):
        if i != 0:
            if brick_list[i][0][2] - brick_list[i-1][0][2] >= 2:
                difference = brick_list[i][0][2] - brick_list[i-1][0][2] - 1
        brick_list[i] = ((brick_list[i][0][0],brick_list[i][0][1],brick_list[i][0][2]-difference),(brick_list[i][1][0],brick_list[i][1][1],brick_list[i][1][2]-difference))

    return brick_list

In [920]:
def brick_height(brick):
    return brick[1][2]-brick[0][2]

In [921]:
def draw_bricks(brick_list):
    #realise this could make things a lot easier in the poker hands puzzle
    brick_list.sort(key=cmp_to_key(third_brick_compare))

    height_map = {
        [i for i in range(brick_list[-1][0][-1]+1)][j]: [] for j in range(brick_list[-1][0][-1]+1)
    }

    if 0 in height_map.keys():
        del height_map[0]

    for brick in brick_list:
        brick_start_height = brick[0][-1]
        height_map[brick_start_height].append(brick)

    for k in list(height_map.keys())[1:]:

        falling_brick = []
        
        layer = height_map[k-1]
        layer_addition = []
        for j in range(k-1):
            if j in height_map.keys():
                for brick2 in height_map[j]:
                    if brick_height(brick2) == k-1-j:
                        layer_addition.append(brick2)

        for brick in height_map[k]:
            if not is_brick_under(layer+layer_addition,brick):
                falling_brick.append(brick)
        
        for brick in falling_brick:
            height_map[k-1].append(brick)
            height_map[k].remove(brick)

    return height_map

In [922]:
test_height_map = draw_bricks(test_bricks)

In [923]:
test_height_map

{1: [((1, 0, 1), (1, 2, 1))],
 2: [((0, 0, 2), (2, 0, 2)), ((0, 2, 3), (2, 2, 3))],
 3: [((0, 0, 4), (0, 2, 4))],
 4: [((2, 0, 5), (2, 2, 5))],
 5: [((0, 1, 6), (2, 1, 6))],
 6: [],
 7: [((1, 1, 8), (1, 1, 9))],
 8: []}

In [924]:
def can_I_disintegrate(brick, brick_layer, height_map):
    
    index = height_map[brick_layer].index(brick)

    h = brick_height(brick)
    layer_addition = []
    for j in range(brick_layer+h):
        if j in height_map.keys():
            for brick2 in height_map[j]:
                if brick_height(brick2) == brick_layer+h-j:
                    layer_addition.append(brick2)
        

    if brick_height(brick) == 0:
        test_layer = height_map[brick_layer][:index] + height_map[brick_layer][index+1:] + layer_addition
    else:
        layer_addition.remove(brick)
        test_layer = height_map[brick_layer+brick_height(brick)] + height_map[brick_layer+brick_height(brick)] + layer_addition
    
    if brick_layer+brick_height(brick)+1 in height_map.keys():
        for brick in height_map[brick_layer+brick_height(brick)+1]:
            if not is_brick_under(test_layer,brick):
                return False
       
    return True

In [925]:
def part_one(height_map):
    new_height_map = height_map.copy()

    disintegrate_list=[]
    for k,v in list(height_map.items())[:-1]:

        for brick in v:
            if can_I_disintegrate(brick, k, new_height_map):
                disintegrate_list.append(brick)

    disintegrate_list += height_map[len(height_map)-1]

    return disintegrate_list

In [926]:
def check_again_reversed(height_map):

    bricks_fallen = []
    for k in list(height_map.keys())[1:]:

        falling_brick = []

        
        layer = height_map[k-1]
        layer_addition = []
        for j in range(k-1):
            if j in height_map.keys():
                for brick2 in height_map[j]:
                    if brick_height(brick2) == k-1-j:
                        layer_addition.append(brick2)

        for brick in height_map[k]:
            if not is_brick_under(layer+layer_addition,brick):
                falling_brick.append(brick)

        bricks_fallen += falling_brick
        
        for brick in falling_brick:
            height_map[k-1].append(brick)
            height_map[k].remove(brick)

    return height_map, bricks_fallen

def get_the_map_right_reversed(height_map):
    
    old_height_map = copy.deepcopy(height_map)
    new_height_map, bricks_fallen = check_again_reversed(height_map)
    

    
    while new_height_map != old_height_map:

        old_height_map = copy.deepcopy(new_height_map)
        new_height_map, more_bricks_fallen = check_again_reversed(new_height_map)
        bricks_fallen += more_bricks_fallen

    for k,v in new_height_map.items():
        new_height_map[k]=[((brick[0][0], brick[0][1], k), (brick[1][0], brick[1][1], k+brick_height(brick))) for brick in v]
            

    return new_height_map, bricks_fallen

In [927]:
test_height_map

{1: [((1, 0, 1), (1, 2, 1))],
 2: [((0, 0, 2), (2, 0, 2)), ((0, 2, 3), (2, 2, 3))],
 3: [((0, 0, 4), (0, 2, 4))],
 4: [((2, 0, 5), (2, 2, 5))],
 5: [((0, 1, 6), (2, 1, 6))],
 6: [],
 7: [((1, 1, 8), (1, 1, 9))],
 8: []}

In [928]:
test_height_map = draw_bricks(test_bricks)

In [929]:
test_height_map = draw_bricks(test_bricks)
test_height_map = get_the_map_right_reversed(test_height_map)[0]
print(len(part_one(test_height_map)))

5


In [936]:
#print(input_bricks[184:])
input_height_map = draw_bricks(input_bricks)
input_height_map= get_the_map_right_reversed(input_height_map)[0]

#for some reason get_the_map_right doesn't sort the index

#x should be in that bleeding list!!
input_dis_list = part_one(input_height_map)
print(len(input_dis_list))

405


In [940]:
input_height_map[153]

[((6, 0, 153), (6, 2, 153)),
 ((8, 7, 153), (8, 7, 155)),
 ((2, 5, 153), (2, 5, 155)),
 ((5, 1, 153), (5, 4, 153)),
 ((7, 4, 153), (8, 4, 153))]

## Part 2

In [945]:
def how_many_fall_2(height_map, brick, brick_layer):

    new_height_map = copy.deepcopy(height_map)

    new_height_map[brick_layer].remove(brick)

    bricks_fallen = len(set(get_the_map_right_reversed(new_height_map)[1]))

    return bricks_fallen

In [946]:
def part_2_2(height_map,disintegrate_list):

    brick_map = {}

    sum_ = 0
    for k,v in height_map.items():

        for brick in v:
            if brick not in disintegrate_list:
                number_fallen = how_many_fall_2(height_map,brick,k)
                sum_ += number_fallen
                print(brick, sum_)
                brick_map[brick] = number_fallen

    return sum_, brick_map

In [947]:
input_sum_, input_brick_map = part_2_2(input_height_map,input_dis_list)

((1, 9, 1), (3, 9, 1)) 1
((2, 6, 1), (2, 6, 3)) 6
((6, 4, 1), (6, 4, 3)) 1228
((7, 6, 1), (7, 8, 1)) 1230
((9, 4, 1), (9, 4, 2)) 1232
((0, 5, 1), (2, 5, 1)) 1234
((2, 0, 1), (2, 0, 4)) 1246
((1, 3, 1), (1, 3, 4)) 1247
((7, 1, 1), (9, 1, 1)) 1248
((4, 2, 1), (4, 4, 1)) 1252
((0, 8, 1), (2, 8, 1)) 1254
((6, 8, 1), (6, 9, 1)) 1255
((2, 4, 1), (3, 4, 1)) 1266
((8, 9, 1), (9, 9, 1)) 1267
((7, 5, 2), (7, 7, 2)) 1268
((5, 1, 2), (7, 1, 2)) 1270
((2, 3, 2), (5, 3, 2)) 1271
((3, 2, 2), (5, 2, 2)) 1272
((0, 8, 2), (3, 8, 2)) 1273
((0, 5, 2), (0, 7, 2)) 1274
((3, 4, 2), (3, 7, 2)) 1284
((9, 3, 3), (9, 4, 3)) 1285
((6, 1, 3), (7, 1, 3)) 1286
((2, 7, 3), (4, 7, 3)) 1294
((6, 3, 4), (6, 6, 4)) 2515
((2, 4, 4), (2, 6, 4)) 2519
((4, 6, 4), (4, 9, 4)) 2526
((6, 6, 5), (6, 9, 5)) 3706
((2, 2, 5), (2, 4, 5)) 3709
((6, 3, 5), (6, 5, 5)) 3748
((1, 0, 5), (4, 0, 5)) 3759
((4, 7, 5), (4, 7, 5)) 3765
((6, 9, 6), (6, 9, 8)) 4938
((2, 4, 6), (2, 5, 6)) 4940
((3, 0, 6), (3, 2, 6)) 4947
((6, 8, 6), (9, 8, 6)) 495

In [756]:
part_2_2(test_height_map, part_one(test_height_map))

((1, 0, 1), (1, 2, 1)) 6
((0, 1, 6), (2, 1, 6)) 7


7